In [19]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import os


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [20]:
# device = torch.device('cuda' if torch.backends.cuda.is_available() else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps


In [21]:
os.listdir('./../Dataset_Student/unlabeled/video_10000')

['image_15.png',
 'image_14.png',
 'image_16.png',
 'image_17.png',
 'image_13.png',
 'image_12.png',
 'image_10.png',
 'image_11.png',
 'image_8.png',
 'image_9.png',
 'image_2.png',
 'image_3.png',
 'image_1.png',
 'image_0.png',
 'image_4.png',
 'image_5.png',
 'image_7.png',
 'image_6.png',
 'image_20.png',
 'image_21.png',
 'image_19.png',
 'image_18.png']

In [22]:
frames = []

base_dir = './../Dataset_Student/unlabeled/video_10000/'

image_names = [f'image_{i}.png' for i in range(22)]

for file_name in image_names:
    img = plt.imread(base_dir + file_name)
    frames.append(img)
# plt.imshow(frames[3])
frames[3].shape

(160, 240, 3)

In [29]:
class CustomDataset(Dataset):
    def __init__(self, n_videos):
        self.video_idxs = torch.tensor([i for i in range(10000, n_videos+10000)])

    def __len__(self):
        return len(self.video_idxs)

    def __getitem__(self, idx):
#         global net_id
        i = self.video_idxs[idx]
        file_path = f'./../Dataset_Student/unlabeled/video_{i}/'
        x = []
        for j in range(11):
            x.append(torch.tensor(plt.imread(file_path+f'image_{j}.png')).permute(2, 0, 1))
        x = torch.stack(x, 0)
        y = []
        for j in range(11, 22):
            y.append(torch.tensor(plt.imread(file_path+f'image_{j}.png')).permute(2, 0, 1))
        y = torch.stack(y, 0)
        return x, y

In [30]:
batch_size = 8

# Create DataLoader
train_dataset = CustomDataset(3)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [31]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([3, 11, 3, 160, 240]) torch.Size([3, 11, 3, 160, 240])


In [ ]:
# [1, 2, 1, 2]

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_channels, hid_channels):
        super().__init__()
        self.inp_enc = nn.Sequential(nn.Conv2d(in_channels, hid_channels, kernel_size=3, stride=1, padding=1),
                                 nn.GroupNorm(2, hid_channels),
                                 nn.LeakyReLU(0.2))
        self.enc = nn.Sequential(nn.Conv2d(hid_channels, hid_channels, kernel_size=3, stride=2, padding=1),
                                 nn.GroupNorm(2, hid_channels),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(hid_channels, hid_channels, kernel_size=3, stride=1, padding=1),
                                 nn.GroupNorm(2, hid_channels),
                                 nn.LeakyReLU(0.2),
                                 nn.Conv2d(hid_channels, hid_channels, kernel_size=3, stride=2, padding=1),
                                 nn.GroupNorm(2, hid_channels),
                                 nn.LeakyReLU(0.2))
    def forward(self, x):
        b, t, c, h, w = x.shape
        x = self.inp_enc(x.view(b*t, c, h, w))
        x = self.inp_enc(x)
        res = x.clone()
        x = self.enc(x)
        return x, res